<a href="https://colab.research.google.com/github/kazumah1/resnet-traffic-classification/blob/main/traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import kagglehub
import os
from IPython.display import Image, display
import pandas as pd
import numpy as np

In [ ]:
import kagglehub

path = kagglehub.dataset_download("owm4096/street-objects")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'street-objects' dataset.
Path to dataset files: /kaggle/input/street-objects


In [ ]:
print(os.listdir(path))

['dataset', 'paths.csv']


In [ ]:
import json
labels_path = path + '/paths.csv'
labels_df = pd.read_csv(labels_path)
labels_df.rename({"Unnamed: 0": "i"}, axis='columns', errors="raise", inplace=True)
print(labels_df)

         i                                              image  label
0        0  /kaggle/input/street-objects/dataset/bicycle/1...      0
1        1  /kaggle/input/street-objects/dataset/bicycle/1...      0
2        2  /kaggle/input/street-objects/dataset/bicycle/1...      0
3        3  /kaggle/input/street-objects/dataset/bicycle/1...      0
4        4  /kaggle/input/street-objects/dataset/bicycle/1...      0
...    ...                                                ...    ...
9874  9874  /kaggle/input/street-objects/dataset/truck/987...      6
9875  9875  /kaggle/input/street-objects/dataset/truck/987...      6
9876  9876  /kaggle/input/street-objects/dataset/truck/987...      6
9877  9877  /kaggle/input/street-objects/dataset/truck/996...      6
9878  9878  /kaggle/input/street-objects/dataset/truck/997...      6

[9879 rows x 3 columns]


In [ ]:
from torchvision.models import resnet18
import torch
from torch import tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.io import read_image
import torchvision.transforms.v2 as v2
import torch.nn as nn
import torch.optim as optim

In [ ]:
class ImageDataset(Dataset):
  def __init__(self, df, transform):
    self.df = df
    self.transform = transform
  def __len__(self):
    return len(self.df)
  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    image = read_image(row['image'])
    if self.transform:
      image = self.transform(image)
    label = tensor([row['label']], dtype=torch.long)
    return (image, label)

In [ ]:
model = resnet18()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.fc = nn.Linear(model.fc.in_features, 7)
model.to(device)
print(device)


cpu


In [ ]:
transform = v2.Compose([
                        v2.Resize((224, 224)),
                        v2.ToDtype(torch.float32, scale=True)
                        ])
dataset = ImageDataset(labels_df, transform)

In [ ]:
generator = torch.Generator().manual_seed(42)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(len(train_dataset))
print(len(val_dataset))

7903
1976


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
def train(lr=0.001, epochs=8, verbose=True, from_base=True):
  if from_base:
    model = resnet18()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.fc = nn.Linear(model.fc.in_features, 7)
    model.to(device)


  criterion = nn.CrossEntropyLoss()

  optimizer = optim.Adam(model.parameters(), lr=lr)
  num_epochs = 8

  train_losses = []
  val_losses = []
  val_accuracies = []

  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      labels = labels.squeeze(-1)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
      for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        labels = labels.squeeze(-1)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(100 * correct / total)

    if verbose:
      print(f"Epoch {epoch+1} | Train Loss: {train_losses[-1]:.4f} | "
            f"Val Loss: {val_losses[-1]:.4f} | Val Accuracy: {val_accuracies[-1]:.2f}%")
  if verbose:
    print(f"Learning rate: {lr} | Val Accuracy: {val_accuracies}")

  torch.save(model.state_dict(), f'{lr}_resnet.pth')
  return train_losses, val_losses, val_accuracies, model

In [ ]:
train_ls = []
val_ls = []
accs = []
lrs = [0.01, 0.005, 0.001, 0.0005, 0.0001]
for lr in lrs:
  train_losses, val_losses, val_accuracies, curr_model = train(lr=lr, epochs=3)
  train_ls.append(train_losses)
  val_ls.append(val_losses)
  accs.append(val_accuracies)

Epoch 1 | Train Loss: 1.5313 | Val Loss: 1.4205 | Val Accuracy: 50.56%


In [ ]:
import matplotlib.pyplot as plt

# Plot
plt.figure(figsize=(10, 6))
for i in range(len(lrs)):
    plt.plot(range(1, 9), accs[i], marker='o', label=f"LR = {lrs[i]}")

plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy (%)")
plt.title("Learning Rate Tuning Results")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# 🔴 Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss', marker='o')
plt.plot(val_losses, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# 🟢 Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy (%)', color='green', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Collect all true and predicted labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = curr_model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy().flatten())


# Generate confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['bicycle', 'car', 'limit30', 'person', 'stop', 'trafficlight', 'truck'])

print("Model output shape:", outputs.shape)
print("labels:", np.unique(all_labels))
print("preds: ", np.unique(all_preds))
# Plot
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title(" Confusion Matrix - Validation Set")
plt.grid(False)
plt.show()